In [5]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
#Load the Dataset
dataset = pd.read_csv('dataset.csv')

C:\Users\Babayeju\AppData\Local\Temp\ipykernel_32224\1744678645.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('dataset.csv')


In [4]:
# Data summary
data_summary = {
    "Total Rows": dataset.shape[0],
    "Total Columns": dataset.shape[1],
    "Column Types": dataset.dtypes.value_counts().to_dict()
}

# Missing values analysis
missing_values = dataset.isnull().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0]  # Only show columns with missing values

# Statistical summary for numerical features
statistical_summary = dataset.describe()

data_summary, missing_values.head(10), statistical_summary


({'Total Rows': 100000,
  'Total Columns': 82,
  'Column Types': {dtype('int64'): 68, dtype('float64'): 8, dtype('O'): 6}},
 SURVEY_AVG_NPS_SCORE          99635
 SURVEY_AVG_CXI_SCORE          98810
 SURVEY_AVG_CASE_MOOD_SCORE    97556
 DNB_GLOBAL_SALES_REVENUE      38137
 DNB_GLOBAL_EMPLOYEE_COUNT     38137
 SECTOR                        15070
 dtype: int64,
        TENURE_MONTHS  DNB_GLOBAL_SALES_REVENUE  DNB_GLOBAL_EMPLOYEE_COUNT  \
 count  100000.000000              6.186300e+04               6.186300e+04   
 mean       43.069110              3.049351e+09               7.569068e+03   
 std        33.474739              2.298173e+10               7.891647e+04   
 min        -1.000000              1.000000e+00               2.000000e+00   
 25%        21.000000              1.285110e+07               7.900000e+01   
 50%        35.000000              5.596105e+07               2.600000e+02   
 75%        52.000000              2.812486e+08               1.092000e+03   
 max       191.

# Initial Obeservations
## Top 5 Columns with Missing Values:
1. **SURVEY_AVG_NPS_SCORE**: 99,635 missing
2. **SURVEY_AVG_CXI_SCORE**: 98,810 missing
3. **SURVEY_AVG_CASE_MOOD_SCORE**: 97,556 missing
4. **DNB_GLOBAL_SALES_REVENUE**: 38,137 missing
5. **DNB_GLOBAL_EMPLOYEE_COUNT**: 38,137 missing

## Statistical Summary (Key Points)
- **TENURE_MONTHS**: Varies widely, with a minimum of -1 (which might be an error) and a maximum of 191 months.
- **CHURN**: The target variable, with only 0.67% churned customers, indicating a highly imbalanced dataset.

## Imbalanced Dataset
- **Low churn rate**: Suggests class(categories) are not represented equally. This Needs addressing during model training.

## Missing Data
- **High levels of missing data**: Particularly in survey-related scores, requiring careful handling.

## Potential Data Issues
- **Negative values in TENURE_MONTHS and CONTRACT_LENGTH**: May indicate data entry errors.

## Diverse Feature Set
- **Mix of customer profile, usage, and engagement data**: Will help with the churn prediction.



## Next Steps

1. **Data Cleaning**: Address missing values, incorrect data entries, and potential outliers.
2. **Exploratory Data Analysis (EDA)**: Conduct deeper EDA to understand the relationship between individual features and the target variable.
3. **Feature Engineering**: Create new features that could be more predictive of churn.


# Data Cleaning

In [6]:
from sklearn.impute import SimpleImputer
import numpy as np

# Handling missing values
# Inputting missing values for DNB_GLOBAL_SALES_REVENUE and DNB_GLOBAL_EMPLOYEE_COUNT with median
median_imputer = SimpleImputer(strategy='median')
dataset['DNB_GLOBAL_SALES_REVENUE'] = median_imputer.fit_transform(dataset[['DNB_GLOBAL_SALES_REVENUE']])
dataset['DNB_GLOBAL_EMPLOYEE_COUNT'] = median_imputer.fit_transform(dataset[['DNB_GLOBAL_EMPLOYEE_COUNT']])

# For survey scores with a high percentage of missing values, we'll impute with a default value indicating no response
# This approach retains the information that these surveys were not filled
dataset['SURVEY_AVG_NPS_SCORE'].fillna(-1, inplace=True)
dataset['SURVEY_AVG_CXI_SCORE'].fillna(-1, inplace=True)
dataset['SURVEY_AVG_CASE_MOOD_SCORE'].fillna(-1, inplace=True)

# Correcting data inconsistencies
# Fixing negative values in TENURE_MONTHS and CONTRACT_LENGTH by replacing them with NaN and then replacing those NaN values with the median_inputer.
dataset['TENURE_MONTHS'] = dataset['TENURE_MONTHS'].apply(lambda x: np.nan if x < 0 else x) # for ever x value less than 0 replace with NaN
dataset['CONTRACT_LENGTH'] = dataset['CONTRACT_LENGTH'].apply(lambda x: np.nan if x < 0 else x) # for ever x value less than 0 replace with NaN
dataset['TENURE_MONTHS'] = median_imputer.fit_transform(dataset[['TENURE_MONTHS']])
dataset['CONTRACT_LENGTH'] = median_imputer.fit_transform(dataset[['CONTRACT_LENGTH']])

# Convert date columns to datetime format
dataset['ACCOUNTING_MONTH'] = pd.to_datetime(dataset['ACCOUNTING_MONTH'])
dataset['RENEWAL_MONTH'] = pd.to_datetime(dataset['RENEWAL_MONTH'], utc=True)
dataset['CONTRACT_START_DATE'] = pd.to_datetime(dataset['CONTRACT_START_DATE'], utc=True)

# Check the dataset after these operations
dataset.head()


,SALESFORCEACCOUNTID,ACCOUNTING_MONTH,RENEWAL_MONTH,CONTRACT_START_DATE,REGION,SECTOR,TENURE_MONTHS,DNB_GLOBAL_SALES_REVENUE,DNB_GLOBAL_EMPLOYEE_COUNT,CONTRACT_LENGTH,...,SUM_HIGH_CASES,SUM_MEDIUM_CASES,SUM_LOW_CASES,SUM_STANDARD_CASES,SUPPORT_CASE_NUMBEROFSLABREACHES,BACKLOG,SURVEY_AVG_CXI_SCORE,SURVEY_AVG_NPS_SCORE,SURVEY_AVG_CASE_MOOD_SCORE,CHURN
0,0011N00001hBZM7QAO,2021-10-01 00:00:00+00:00,2021-12-01 00:00:00+00:00,2020-12-24 00:00:00+00:00,UKI,Professional Services,126.0,55961051.0,260.0,364.0,...,0,0,0,0,0,1,-1.0,-1.0,-1.0,0
1,0011N00001hBZM7QAO,2021-11-01 00:00:00+00:00,2021-12-01 00:00:00+00:00,2020-12-24 00:00:00+00:00,UKI,Professional Services,127.0,55961051.0,260.0,364.0,...,0,0,0,0,0,1,-1.0,-1.0,-1.0,0
2,0011N00001hBZM7QAO,2021-12-01 00:00:00+00:00,2022-12-01 00:00:00+00:00,2021-12-24 00:00:00+00:00,UKI,Professional Services,128.0,55961051.0,260.0,364.0,...,0,0,0,0,0,1,-1.0,-1.0,-1.0,0
3,0011N00001hBZM7QAO,2022-01-01 00:00:00+00:00,2022-12-01 00:00:00+00:00,2021-12-24 00:00:00+00:00,UKI,Professional Services,129.0,55961051.0,260.0,364.0,...,0,0,0,0,0,1,-1.0,-1.0,-1.0,0
4,0011N00001hBZM7QAO,2022-02-01 00:00:00+00:00,2022-12-01 00:00:00+00:00,2021-12-24 00:00:00+00:00,UKI,Professional Services,130.0,55961051.0,260.0,364.0,...,0,0,0,0,0,1,-1.0,-1.0,-1.0,0


## Handling Missing Values

- **High Percentage Missing Values**: 
  - Columns like survey scores with a high percentage of missing values need a decision on whether to impute, drop, or transform.
- **Low Percentage Missing Values**:
  - For columns with fewer missing values, consider imputation using mean, median, or mode. Could potentially use a more sophisticated method like k-NN imputation (however for this we will use our basic statistics for missing vlaues).
- **Specific Replacement Strategies**:
    - `DNB_GLOBAL_SALES_REVENUE` and `DNB_GLOBAL_EMPLOYEE_COUNT` have missing data were replaced using median values.
  - **Survey Scores**:
    - `SURVEY_AVG_NPS_SCORE`, `SURVEY_AVG_CXI_SCORE`, `SURVEY_AVG_CASE_MOOD_SCORE` all have High missing values replaced with a default value of -1, indicating no response.

## Correcting Data Inconsistencies

- **Negative Values**:
  - Address negative values in `TENURE_MONTHS` and `CONTRACT_LENGTH`.
- **Date Columns**:
  - Convert date columns to datetime format and extract information such as month and year.


## Outlier Detection and Handling

- **Identification**:
  - Identify outliers in numerical columns.
- **Handling Strategy**:
  - Decide on a strategy for handling outliers, such as capping or transformation.
  - Specific columns like `ACCOUNTING_MONTH`, `RENEWAL_MONTH`, and `CONTRACT_START_DATE` were converted to datetime format for better handling and feature engineering.


# Feature Engineering